In [1]:
# Initiate linear demand curve parameters for simulation analysis
# One linear demand curve is initiated for each rate class, arrival day of week and 
# los combination

In [2]:
import itertools
import numpy as np
from scipy.stats import norm
from scipy.stats import poisson
from sklearn.linear_model import LinearRegression

In [3]:
# Consider two rate classes for simplicity
n_class = 2
# Consider Length-of-Stay (los) of upto three days
los = 3
capacity = 50
# Consider two demand scenario, by changing capacity
demand_intensity = 0.9
# Slope for linear demand curve
slope = np.array([-0.1, -0.15])
# Nightly rates for Monday stay night through Sunday stay night
# Higher weekday rates imply business hotel settings and higher weekend rates imply resorts
rates_init = np.array([[135, 135, 135, 135, 135, 108, 108],
                       [115, 115, 115, 115, 115, 92, 92]])

In [4]:
# Take the slope of -0.1 for rate class 1, then the mean true demand point for any arrival date/lengthof-
# stay combination is passed through by the line defined by this slope; 30 points on the price 
# axis for each arrival date/length-of-stay combination over the first 21 simulation days 
# are randomly generated from the interval (init_rate-50, init_rate+50)
# Sunday-one night stay as an example
drawsize = 30
# Random draw for one night stay for each day of week
randomRates_one = [np.random.randint(rates_init[i, j]-50, rates_init[i, j]+50+1, drawsize) 
                     for i, j in itertools.product(range(n_class), range(7))]
randomRates_one = np.array(randomRates_one).reshape(n_class, 7, drawsize)
# Check the averages are close to rates_init
np.round(np.average(randomRates_one, axis = 2), decimals=1)

array([[135.1, 139.2, 142.7, 144.7, 138.3, 104.9,  98.9],
       [124.1, 113.4, 118.2, 121.8, 110.9,  91.3,  96.6]])

In [5]:
randomRates_one.shape

(2, 7, 30)

In [6]:
# Construct rates for more than one night stay, up to 3-night stay
# Sunday night arrival, two night stay rates equal Sunday one night + Monday one night stay
randomRates_two = [randomRates_one[i, j%7] 
                     + randomRates_one[i, (j+1)%7] 
                     for i, j in itertools.product(range(n_class), range(7))]

randomRates_three = [randomRates_one[i, j%7] 
                       + randomRates_one[i, (j+1)%7] 
                       + randomRates_one[i, (j+2)%7] 
                     for i, j in itertools.product(range(n_class), range(7))]

In [7]:
# For each arrival date, los can be 1-night, 2-night, or 3-night.
# Accordingly, the rates will be corresponding to different los.
# e.g., for Mon arrival, 2-night stay, the rate equals Mon single night rate + Tu single night rate
randomRates_one = np.array(randomRates_one).reshape(n_class, 7, drawsize)
randomRates_two = np.array(randomRates_two).reshape(n_class, 7, drawsize)
randomRates_three = np.array(randomRates_three).reshape(n_class, 7, drawsize)
randomRates = [(randomRates_one[i, j], randomRates_two[i, j], randomRates_three[i, j]) 
               for i, j in itertools.product(range(n_class), range(7))]

In [8]:
# axis 0 = n_class, axis 1 = day of week, axis 2 = los, and axis 3 = random draws
randomRates = np.array(randomRates).reshape(n_class, 7, los, drawsize)
randomRates.shape

(2, 7, 3, 30)

In [9]:
# Randomly generated rates for class 1, Sunday arrivals
randomRates[0, 0]

array([[127, 180,  91, 172, 139, 160, 169, 115,  94, 122, 174,  96, 130,
        146, 128, 148, 109, 156, 109,  92, 110, 100,  86, 183, 125, 158,
        161, 132, 163, 179],
       [289, 271, 268, 349, 248, 269, 354, 274, 241, 294, 284, 219, 292,
        331, 233, 316, 206, 255, 274, 210, 290, 226, 202, 295, 261, 286,
        297, 292, 263, 340],
       [458, 418, 429, 478, 390, 369, 499, 385, 382, 460, 449, 352, 387,
        512, 412, 447, 374, 360, 446, 303, 385, 391, 373, 428, 428, 424,
        481, 451, 378, 461]])

In [10]:
# Check accuracy, result should be similar to init_rate
np.mean(randomRates[1], axis = 2)

array([[124.06666667, 237.5       , 355.73333333],
       [113.43333333, 231.66666667, 353.5       ],
       [118.23333333, 240.06666667, 351.        ],
       [121.83333333, 232.76666667, 324.03333333],
       [110.93333333, 202.2       , 298.76666667],
       [ 91.26666667, 187.83333333, 311.9       ],
       [ 96.56666667, 220.63333333, 334.06666667]])

In [11]:
# Construct rates for more than one night stay, up to 3-night stay
rates_init

array([[135, 135, 135, 135, 135, 108, 108],
       [115, 115, 115, 115, 115,  92,  92]])

In [12]:
# Calculate averate rates for each arrival day of week and los combination
rates_arrival_los = [[rates_init[i, j],
                      rates_init[i, j] + rates_init[i, (j+1)%7],
                      rates_init[i, j] + rates_init[i, (j+1)%7] + rates_init[i, (j+2)%7]] 
                      for i, j in itertools.product(range(n_class), range(7))]
# Store it as a numpy array
rates_arrival_los = np.array(rates_arrival_los).reshape(n_class, 7, los)

In [13]:
rates_arrival_los

array([[[135, 270, 405],
        [135, 270, 405],
        [135, 270, 405],
        [135, 270, 378],
        [135, 243, 351],
        [108, 216, 351],
        [108, 243, 378]],

       [[115, 230, 345],
        [115, 230, 345],
        [115, 230, 345],
        [115, 230, 322],
        [115, 207, 299],
        [ 92, 184, 299],
        [ 92, 207, 322]]])

In [14]:
# Calculate y-intercepts, assuming los distribution for a stay night is 1/3, 1/3, and 1/3
# There exist one 1-night stay, two 2-night stays and three 3-night stays that cover 
# stay night in question.
# Each rate class contribute half of the demand for each stay night
half_demand = 0.5 * capacity * demand_intensity
# Intercepts for 1-night stay
A1 = np.array([[1, 0, 0, 0, 0, 0, 0],
               [0, 1, 0, 0, 0, 0, 0],
               [0, 0, 1, 0, 0, 0, 0],
               [0, 0, 0, 1, 0, 0, 0],
               [0, 0, 0, 0, 1, 0, 0],
               [0, 0, 0, 0, 0, 1, 0],
               [0, 0, 0, 0, 0, 0, 1]])
b1 = [(half_demand * 1/3 - slope[i] * rates_arrival_los[i, j, 0]) 
               for i, j in itertools.product(range(n_class), range(7))]
b1 = np.array(b1).reshape(n_class, 7)
x1 = [np.linalg.solve(A1, b1[i]) for i in range(n_class)]

# Intercepts for 2-night stay
A2 = np.array([[1, 0, 0, 0, 0, 0, 1],
               [1, 1, 0, 0, 0, 0, 0],
               [0, 1, 1, 0, 0, 0, 0],
               [0, 0, 1, 1, 0, 0, 0],
               [0, 0, 0, 1, 1, 0, 0],
               [0, 0, 0, 0, 1, 1, 0],
               [0, 0, 0, 0, 0, 1, 1]])
b2 = [(half_demand * 1/3 - slope[i] * (rates_arrival_los[i, j, 1]
                                               +rates_arrival_los[i, (j-1+7)%7, 1])) 
               for i, j in itertools.product(range(n_class), range(7))]
b2 = np.array(b2).reshape(n_class, 7)
x2 = [np.linalg.solve(A2, b2[i]) for i in range(n_class)]

# Intercepts for 3-night stay
A3 = np.array([[1, 0, 0, 0, 0, 1, 1],
               [1, 1, 0, 0, 0, 0, 1],
               [1, 1, 1, 0, 0, 0, 0],
               [0, 1, 1, 1, 0, 0, 0],
               [0, 0, 1, 1, 1, 0, 0],
               [0, 0, 0, 1, 1, 1, 0],
               [0, 0, 0, 0, 1, 1, 1]])
b3 = [(half_demand * 1/3 - slope[i] * (rates_arrival_los[i, j, 2]
                                               +rates_arrival_los[i, (j-1+7)%7, 2]
                                               +rates_arrival_los[i, (j-2+7)%7, 2]))
             for i, j in itertools.product(range(n_class), range(7))]
b3 = np.array(b3).reshape(n_class, 7)
x3 = [np.linalg.solve(A3, b3[i]) for i in range(n_class)]

In [15]:
# Create intercepts matrix for all arrival day-los combinations
# Total 2 * 7 * 3 = 42 intercepts
intercepts = np.concatenate((np.array(x1), np.array(x2), np.array(x3)), axis=1)
intercepts = np.reshape(intercepts, (n_class, los, 7), order='C')
intercepts = np.array([np.transpose(intercepts[i]) for i in range(n_class)])

In [16]:
intercepts

array([[[21.  , 30.75, 43.  ],
        [21.  , 30.75, 43.  ],
        [21.  , 30.75, 43.  ],
        [21.  , 30.75, 40.3 ],
        [21.  , 28.05, 37.6 ],
        [18.3 , 25.35, 37.6 ],
        [18.3 , 28.05, 40.3 ]],

       [[24.75, 38.25, 54.25],
        [24.75, 38.25, 54.25],
        [24.75, 38.25, 54.25],
        [24.75, 38.25, 50.8 ],
        [24.75, 34.8 , 47.35],
        [21.3 , 31.35, 47.35],
        [21.3 , 34.8 , 50.8 ]]])

In [17]:
# Mean of Poisson demand that is generated by the slope and y-intercept
# One mean demand for each of the 30 randomly drawn price
meanDemand = [intercepts[i,j, k] + slope[i] * randomRates[i, j, k, l] 
                    for i, j, k, l in itertools.product(range(n_class), range(7), range(los), range(drawsize))]
meanDemand = np.array(meanDemand).reshape(n_class, 7, los, drawsize)
zeros = np.zeros(n_class * 7 * los * drawsize).reshape(n_class, 7, los, drawsize)
# Can't have megative demand, so truncate mean demand at zero
meanDemand = np.maximum(meanDemand, zeros)

In [18]:
# Mean demand rate for a Sunday arrival (including 1-, 2-, and 3-night stay) 
np.average(meanDemand[0, 0], axis=1)

array([7.48666667, 3.82833333, 2.67333333])

In [19]:
# Mean demand rate for a Sunday arrival (including 1-, 2-, and 3-night stay) 
meanDemand[0, 0, :, 0]

array([8.3 , 1.85, 0.  ])

In [20]:
# Check total mean demand for each control day of week for each random drawn price
totalDemand = [np.sum(meanDemand[i, j,:,k]) 
                     + np.sum(meanDemand[i, (j-1+7)%7, 1:, k]) 
                     + np.sum(meanDemand[i, (j-2+7)%7, 2:, k]) 
                     for i, j, k in itertools.product(range(n_class), range(7), range(drawsize))]
# Check total mean demand for each day of week
# This mean demand should be around half of the expected demand
totalDemand = np.array(totalDemand).reshape(n_class, 7, drawsize)
np.mean(totalDemand, axis=2)

array([[27.03166667, 23.33166667, 20.15833333, 19.74166667, 23.31333333,
        28.07      , 30.66666667],
       [22.7       , 25.95166667, 25.30166667, 26.085     , 31.64833333,
        30.83666667, 25.79166667]])

In [21]:
# Generate random demand for each day of week and length of stay combination
randomDemand = [poisson.rvs(mu, size=1) for mu in np.nditer(meanDemand)]
randomDemand = np.array(randomDemand).reshape(n_class, 7, los, drawsize)

In [22]:
# Class 1 Sunday arrivals for 1-night, 2-night, and 3-night stays
randomDemand[0, 0]

array([[ 6,  7, 13,  3,  6,  3,  5,  4, 11, 13,  5,  8,  5,  6,  9,  5,
         8,  4, 12, 12, 11, 14, 10,  3, 14,  4,  7,  7,  2,  1],
       [ 5,  3,  2,  0,  7,  5,  0,  2,  5,  1,  3,  7,  0,  0, 14,  0,
        11,  4,  2,  9,  2,  9, 15,  0,  4,  3,  1,  0,  5,  0],
       [ 0,  2,  0,  0,  4,  7,  0,  6,  7,  0,  0,  6,  7,  0,  2,  0,
         5, 10,  0, 14,  6,  6,  4,  0,  0,  2,  0,  0,  6,  0]])

In [23]:
# Check number of total arrivals span a stay night in question for each day of week, 
# Consider up to 3-night stay
arrivals = [np.sum(randomDemand[i, j,:,k]) 
            + np.sum(randomDemand[i, (j-1+7)%7, 1:, k]) 
            + np.sum(randomDemand[i, (j-2+7)%7, 2:, k]) 
            for i, j, k in itertools.product(range(n_class), range(7), range(drawsize))]
arrivals = np.array(arrivals).reshape(n_class, 7, drawsize)
np.mean(arrivals, axis = 2)

array([[27.16666667, 24.33333333, 19.66666667, 19.43333333, 23.73333333,
        28.66666667, 30.3       ],
       [23.46666667, 26.4       , 26.56666667, 25.73333333, 30.33333333,
        30.03333333, 26.56666667]])

In [24]:
arrivals.shape

(2, 7, 30)

In [25]:
# Rates and demand for Sunday stay night for rate class 1 
randomRates[0, 0], randomDemand[0, 0]

(array([[127, 180,  91, 172, 139, 160, 169, 115,  94, 122, 174,  96, 130,
         146, 128, 148, 109, 156, 109,  92, 110, 100,  86, 183, 125, 158,
         161, 132, 163, 179],
        [289, 271, 268, 349, 248, 269, 354, 274, 241, 294, 284, 219, 292,
         331, 233, 316, 206, 255, 274, 210, 290, 226, 202, 295, 261, 286,
         297, 292, 263, 340],
        [458, 418, 429, 478, 390, 369, 499, 385, 382, 460, 449, 352, 387,
         512, 412, 447, 374, 360, 446, 303, 385, 391, 373, 428, 428, 424,
         481, 451, 378, 461]]),
 array([[ 6,  7, 13,  3,  6,  3,  5,  4, 11, 13,  5,  8,  5,  6,  9,  5,
          8,  4, 12, 12, 11, 14, 10,  3, 14,  4,  7,  7,  2,  1],
        [ 5,  3,  2,  0,  7,  5,  0,  2,  5,  1,  3,  7,  0,  0, 14,  0,
         11,  4,  2,  9,  2,  9, 15,  0,  4,  3,  1,  0,  5,  0],
        [ 0,  2,  0,  0,  4,  7,  0,  6,  7,  0,  0,  6,  7,  0,  2,  0,
          5, 10,  0, 14,  6,  6,  4,  0,  0,  2,  0,  0,  6,  0]]))

In [26]:
# Create linear regression object
slopes_update = []
intercepts_update = []
regr = LinearRegression()
for i, j, k in itertools.product(range(n_class), range(7), range(los)):
    regr.fit(randomRates[i, j, k, :].reshape(drawsize, -1), randomDemand[i, j, k, :])
    intercepts_update.append(regr.intercept_)
    slopes_update.append(regr.coef_)

In [27]:
slopes_update = np.array(slopes_update).reshape(n_class, 7, los)
intercepts_update = np.array(intercepts_update).reshape(n_class, 7, los)

In [28]:
slopes_update = np.round(slopes_update, 2)
intercepts_update = np.round(intercepts_update, 0)

In [29]:
slopes_update, intercepts_update

(array([[[-0.1 , -0.09, -0.07],
         [-0.11, -0.07, -0.06],
         [-0.1 , -0.07, -0.06],
         [-0.11, -0.06, -0.09],
         [-0.1 , -0.1 , -0.06],
         [-0.1 , -0.1 , -0.09],
         [-0.11, -0.08, -0.07]],
 
        [[-0.17, -0.09, -0.09],
         [-0.14, -0.11, -0.07],
         [-0.14, -0.07, -0.09],
         [-0.13, -0.1 , -0.09],
         [-0.16, -0.09, -0.08],
         [-0.12, -0.12, -0.09],
         [-0.17, -0.12, -0.09]]]),
 array([[[20., 28., 32.],
         [23., 24., 28.],
         [20., 23., 26.],
         [23., 21., 38.],
         [22., 29., 23.],
         [18., 25., 35.],
         [19., 24., 30.]],
 
        [[27., 24., 35.],
         [24., 31., 29.],
         [24., 21., 36.],
         [22., 28., 34.],
         [25., 25., 27.],
         [18., 27., 31.],
         [24., 30., 34.]]]))